<center><h1>Py-Galaxies examples</h1></center>

# SUB-HALO Stars

## Import modules

In [1]:
import PyGalaxies
import time
import numpy as np

#
model_param_filepath='Input_Params/input_params.yml'
debug_flag = False
verbosity = 1 
time_code = True

## Get model params and print

In [2]:
model_params = PyGalaxies.ModelParams(model_param_filepath,verbosity,debug_flag)
model_params.output_params() 

input_files         : {'graph_file': '../Input Data/mega_graph_latest.hdf5'}
output_files        : {'halo_file': '../Output Data/SMT13HaloOutput.hdf5', 'galaxy_file': '../Output Data/SMT13GalaxyOutput.hdf5'}
cosmology           : {'omega_m': {'Description': 'Matter density parameter', 'Value': 0.3, 'Units': 'None'}, 'f_baryon': {'Description': 'Baryon fraction', 'Value': 0.155, 'Units': 'None'}}
model_switches      : {'HOD': {'Description': 'Halo occupation description for stars (as primitive test)', 'Value': True}, 'sub_halo': {'Description': 'For each graph, load in Sub-halo data?', 'Value': True}}
performance         : {'io_nRec': {'Description': 'Size of HDF5 io buffer (number of records)', 'Value': 1000}}


## Run main routine

In [3]:
# Open the input HDF5 file containing graph groups and create an output file.
HDF_properties = PyGalaxies.HDFProperties(model_params)



start_time = time.time()


for graph_ID in range(0,HDF_properties.no_of_graphs)[:]:

    if verbosity >= 2: print('Processing graph',graph_ID)
    

    graph_properties = PyGalaxies.GraphProperties(graph_ID,
                                       HDF_properties.graph_input_file,
                                       model_params,
                                       HDF_properties.part_mass)
    
    
    array_of_halo_properties = np.empty(len(graph_properties.graph_halo_ids),
                                        dtype = object)
    
    if type(graph_properties.n_subhalos) == int:
        continue
        
    for snap_ID in graph_properties.generation_id:
        

        if graph_properties.generation_start_index[snap_ID] == 2**30:
            continue
    
        
        this_generation_halo = graph_properties.graph_halo_ids[
                               graph_properties.generation_start_index[snap_ID]:
                               graph_properties.generation_start_index[snap_ID] + 
                               graph_properties.generation_length[snap_ID]]
        
        for halo_ID in this_generation_halo:
            
            halo = PyGalaxies.HaloProperties(str(graph_ID),snap_ID,halo_ID,
                                  graph_properties,HDF_properties.part_mass)
        
        
        
            PyGalaxies.inclusive_mass_contribution(halo)
            
            
            if halo.nprog > 0: 
                PyGalaxies.gather_progenitors(halo,HDF_properties.part_mass)
            else:
                halo.mass_from_progenitors = 0.
            
            PyGalaxies.set_baryon_fraction(halo,array_of_halo_properties,model_params.f_baryon)
            
            
            
            halo.extract_sub_halo_data(graph_properties,
                           HDF_properties.dtype_subhalo_stores,
                           HDF_properties.part_mass,
                           model_params.f_baryon)

            halo.done=True
            
            array_of_halo_properties[halo_ID] = halo

            HDF_properties.n_halo +=1 

    # Save output
    HDF_properties.output_halos(array_of_halo_properties)




# Close input file
HDF_properties.close_graph_io(HDF_properties.graph_input_file)


# Close output files
if HDF_properties.halo_output_iRec > 0: 

    halo_output_iRec = HDF_properties.flush_output()
    
HDF_properties.close_graph_io(HDF_properties.halo_output_file)


end_time = time.time()
print('This took {} seconds'.format(end_time-start_time))

This took 29.38920021057129 seconds


In [14]:
vars(array_of_halo_properties[13])

{'graph_ID': '975',
 'snap_ID': 52,
 'halo_ID': 13,
 'catalog_ID': 68056,
 'mass': 37242965349.419556,
 'nprog': 1,
 'prog_start': 12,
 'prog_end': 13,
 'prog_ids': array([12]),
 'prog_mass': array([3.72429653e+10]),
 'ndesc': 1,
 'desc_start': 13,
 'desc_end': 14,
 'desc_ids': array([14]),
 'desc_mass': array([3.72429653e+10]),
 'mean_pos': array([34.61936174, 50.70175716, 36.33231109]),
 'velocity_dispersion_3D': 50.951448661215544,
 'mean_vel': array([-351.37498256,   64.4213341 ,  159.75099727]),
 'rms_radius': 0.03572066579517957,
 'mass_baryon': 5772659629.160031,
 'mass_from_progenitors': 4.9537088143502344e+19,
 'mass_baryon_from_progenitors': 0.0,
 'inclusive_contribution': array([1.]),
 'done': True,
 'subhalo_start_index': 0,
 'n_subhalo': 1,
 'sub_graph_halo_ids': array([0]),
 'sub_halo_attrs': array((0, 0.21104259, 2.7932224e+10, 0.),
       dtype=[('central_sub_halo_ID', '<i4'), ('central_sub_halo_dist', '<f8'), ('central_sub_halo_mass', '<f8'), ('prog_contrib_mass', '<f8

In [5]:
# # Open the input HDF5 file containing graph groups and create an output file.
# HDF_properties = PyGalaxies.HDFProperties(model_params)

# start_time = time.time()

# for graph_ID in range(0,HDF_properties.no_of_graphs)[:]:
    
#     if verbosity >= 2: print('Processing graph',graph_ID)
    

#     graph_properties = PyGalaxies.GraphProperties(graph_ID,
#                                        HDF_properties.graph_input_file,
#                                        model_params,
#                                        HDF_properties.part_mass)
    
    
#     array_of_halo_properties = np.empty(len(graph_properties.graph_halo_ids),
#                                         dtype = object)

    
#     # This will be removed when Will updated the files. Just skips over any
#     # graphs that don't have subhalos
#     if type(graph_properties.n_subhalos) == int:
#         continue
        
#     for snap_ID in graph_properties.generation_id:
        

#         if graph_properties.generation_start_index[snap_ID] == 2**30:
#             continue
    
        
#         this_generation_halo = graph_properties.graph_halo_ids[
#                                graph_properties.generation_start_index[snap_ID]:
#                                graph_properties.generation_start_index[snap_ID] + 
#                                graph_properties.generation_length[snap_ID]]
        
#         for halo_ID in this_generation_halo:
            
#             halo = PyGalaxies.HaloProperties(str(graph_ID),snap_ID,halo_ID,
#                                   graph_properties,HDF_properties.part_mass,
#                                   )
        
            
    
  
#             halo.extract_sub_halo_data(graph_properties,
#                                        HDF_properties.dtype_subhalo_stores,
#                                        HDF_properties.part_mass,
#                                        model_params.f_baryon)
                                                            
            
            
# #             if halo.sub_nprog > 0:
# #                 PyGalaxies.sub_gather_progenitors()
        
                    
#             halo.done=True
#             HDF_properties.n_halo +=1 
#             array_of_halo_properties[halo_ID] = halo
            

    
# #     # Save output
# #     HDF_properties.output_halos(array_of_halo_properties)




# # # Close input file
# # HDF_properties.close_graph_io(HDF_properties.graph_input_file)


# # # Close output files
# # if HDF_properties.halo_output_iRec > 0: 

# #     halo_output_iRec = HDF_properties.flush_output()
    
# # HDF_properties.close_graph_io(HDF_properties.halo_output_file)


# end_time = time.time()
# print('This took {} seconds'.format(end_time-start_time))